<a href="https://colab.research.google.com/github/aaanubha/ML_projects/blob/main/News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the dataset
1. id
2. title
3. author
4. text
5. label (1: fake, 0: real)


Importing Dependencies

In [28]:
import numpy as np
import pandas as pd
import re #regular expression: useful for searching text data doc
from nltk.corpus import stopwords #natural lang tool kit
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [7]:
#loading the dataset to a pandas dataframe
dataset = pd.read_csv('/content/train.csv')

In [8]:
dataset.shape

(20800, 5)

In [9]:
#print first 5 rows of df
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
#counting missing values
dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [11]:
#replacing missing values
dataset = dataset.fillna('')

In [12]:
#text is large and takes a lot of time, comparatively title and author is faster
#merging author name and news title
dataset['content'] = dataset['author']+' '+dataset['title'] #plus contactenates 2 strings

In [13]:
print(dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


Stemming

In [18]:
port_stem = PorterStemmer()

In [19]:
def stemming(content): #creating a fun called stemming, content denotes the input
  stemmed_content = re.sub('[^a-zA-Z]',' ', content) #called re lib to search a para for text, sub is used to substitute certain values, ^ excludes everything execpt what is present inside []
  #in our dataset there can be no's and quotes etc, we don't need that
  #if there are nos or commas, they will be replace by the space
  #this process needs to be carried out in content -> authors name and title
  stemmed_content = stemmed_content.lower() #converting everything to lowercase
  stemmed_content = stemmed_content.split() #split and convert to a list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #stemming takes place here
  stemmed_content = ' '.join(stemmed_content) #join all final words
  return stemmed_content

In [20]:
#apply the function to the df
dataset['content'] = dataset['content'].apply(stemming)
#taking content column and applying the stemming function, the column will be replaced

In [21]:
print(dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [22]:
#seperating data and label
x = dataset['content'].values
y = dataset['label'].values

In [23]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [24]:
print(y)

[1 0 1 ... 0 1 1]


Vectorizing data

In [29]:
vectorizer = TfidfVectorizer() #counts the no of times a word is repeating, repetition tells if words is imp or not and accordinly gives a numerical value
#idf stands for inverse freq -> eg. if movie review, then movie name adds new value: this detects that and reduces its value
vectorizer.fit(x)

x = vectorizer.transform(x)

In [30]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Training and testing data

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, stratify = y, random_state = 2)
#y containts 0 and 1, with stratify- 0 and 1 are split almost equally
#random state- used to reproduce a particular code

Model Training: Logistic Regression

In [32]:
model = LogisticRegression()

In [33]:
model.fit(x_train, y_train)

LogisticRegression()

Evaluation

Accuracy score

In [34]:
#accuracy score on the training data
x_train_pred = model.predict(x_train)
td_accuracy = accuracy_score(x_train_pred, y_train)

In [35]:
print("Accuracy on training data: ",td_accuracy)

Accuracy on training data:  0.9865985576923076


In [36]:
#accuracy for test data
x_test_pred = model.predict(x_test)
td_acc = accuracy_score(x_test_pred, y_test)

In [37]:
print("Accuracy on test data: ",td_acc)

Accuracy on test data:  0.9790865384615385


Making a predictive system

In [40]:
x_new = x_test[1]

prediction = model.predict(x_new)
print(prediction)

if (prediction[0] == 0):
  print('News is real')
else:
  print('News is fake')

[0]
News is real
